# Implement A Weighted Frequent Itemset Mining Algorithm for Intelligent Decision in Smart Systems

In [1]:
import numpy as np
import random

# 1. Preliminary

In [2]:
def get_probability_in_transaction(X, T):
    """get the probabilistic of itemset in transaction T

    Args:
        X (list<string>): itemset X
        T (dict<string, double>): transaction T

    Returns:
        prob: the probabilistic of itemset in transaction T
    """
    prob = 1.0
    for i in X:
        prob *= T.get(i, 0.0)
    return prob

def get_weighted_itemset(X, W):
    """get the weight of itemset in weighted table

    Args:
        X (list<string>): itemset X
        W (dict<string, double>): weighted table W

    Returns:
        weight of itemset: the weight of itemset in weighted table
    """
    weighted = 0

    for i in X:
        weighted += W.get(i, 0.0)

    return weighted / len(X)

def compute_support(X, D):
    """compute the support of itemset X

    Args:
        X (list<string>): itemset X
        D (list<dict<string, double>>): Uncertain database

    Returns:
        support: the support of itemset X
    """
    support = 0

    for Ti in D:
        probX= get_probability_in_transaction(X, Ti)
        
        if probX > 0:
            support += 1

    return support

def compute_expected_support(X, D):
    """compute the expected support of itemset X

    Args:
        X (list<string>): itemset X
        D (list<dict<string, double>>): Uncertain database

    Returns:
        expected_support: the expected support of itemset X
    """
    expected_support = 0

    for Ti in D:
        expected_support += get_probability_in_transaction(X, Ti)

    return expected_support

In [3]:
def compute_prWF(X, D, W, min_conf):
    """compute the probabilistic support of itemset X

    Args:
        X (list<string>): itemset X
        D (list<dict<string, double>>): Uncertain database
        W (dict<string, double): Weighted table
        min_conf (double): min confidence

    Returns:
        probabilistic support vector: the probabilistic support vector of itemset X
    """
    convolution_vector = compute_support_probabilistic_vector(X, D)
    weighted = get_weighted_itemset(X, W)
    prWF = 0
    
    for i in range(len(D), 0, -1):
        prWF += convolution_vector[i]*weighted

        if prWF > min_conf:
            return i

    return -1

def compute_support_probabilistic_vector(X, D):
    """calculate support probabilistic vector of itemset X in D 

    Args:
        X (list<string>): itemset X
        D (list<dict<string, double>>): Uncertain database

    Returns:
        support_probabilistic_vector: support probabilistic vector of itemset X in D 
    """
    n = len(D)
    
    # Base case
    if n == 1:
        prob = get_probability_in_transaction(X, D[0])
        f_X = np.zeros(2)
        f_X[0] = 1 - prob
        f_X[1] = prob
        return f_X

    # Recursive case: Partition PDB
    mid = n // 2
    D1 = D[:mid]
    D2 = D[mid:]

    # Recursively call DC on partitions
    f_X1 = compute_support_probabilistic_vector(X, D1)
    f_X2 = compute_support_probabilistic_vector(X, D2)

    # Convolution 2 array
    # Adjust length of f_X1 and f_X2 for convolution
    len_total = len(f_X1) + len(f_X2) - 1
    f_X1 = np.pad(f_X1, (0, len_total - len(f_X1)), mode='constant')
    f_X2 = np.pad(f_X2, (0, len_total - len(f_X2)), mode='constant')

    # Convolution using FFT
    f_X = np.fft.ifft(np.fft.fft(f_X1) * np.fft.fft(f_X2)).real
    f_X = f_X[:n+1]  # Take only the first n+1 elements
    f_X = np.round(f_X, decimals=10)  # Fix numerical inaccuracies

    return f_X

In [4]:
def discover_single_items(D):
    """get all single item in D

    Args:
        D (_type_): Uncertain database

    Returns:
        _type_: list of single item in D
    """
    # Implement discovery of single items using Chernoff-Hoeffding bound
    single_items = set()

    for Ti in D:
        for item in Ti.keys():
            single_items.add(item)

    return single_items

# 2. Algorithms

In [5]:
def Connection(WFIS_prev, CWFIS_1):
    """
    Generate candidate itemsets from the previous level's frequent itemsets and the initial candidate itemsets.
    """
    return {frozenset(a.union(b)) for a in WFIS_prev for b in CWFIS_1 if len(a.union(b)) == len(a) + 1}

def WD_FIM(D, weighted, min_sup, wfpt):
    """
    Implement the custom WD-FIM algorithm.
    """
    WFIS = set()
    
    # Step 1: Initialization
    I = set()
    for transaction in D:
        for item in transaction:
            I.add(frozenset([item]))
    
    # Step 2-7: Scan the dataset and get weighted frequent 1-itemset
    WFIS_1 = set()
    for item in I:
        probabilistic_support = compute_prWF(item, D, weighted, wfpt)
        if  probabilistic_support >= min_sup:
            print(item , "-", probabilistic_support)
            WFIS_1.add(item)
    
    WFIS.update(WFIS_1)
    
    # Step 9-23: Scan the dataset and get weighted frequent k-itemsets
    CWFIS_k = I
    SCWFIS1 = sorted(CWFIS_k, key=lambda x: get_weighted_itemset(x, weighted), reverse=True)
        
    k = 2
    while WFIS_1:
        CWFIS_k_pre = CWFIS_k
        CWFIS_k = Connection(WFIS_1, CWFIS_k)
        NCWFIS_k = Connection(CWFIS_k_pre.difference(WFIS_1), SCWFIS1)
        RCWFIS_k = CWFIS_k.difference(NCWFIS_k)
        
        # print("ck", CWFIS_k)
        # print("nc_k", NCWFIS_k)
        # print("rc_k", RCWFIS_k)
        
        WFIS_k = set()
        for X in RCWFIS_k:
            probabilistic_support = compute_prWF(X, D, weighted, wfpt)
            # print(X, "-",probabilistic_support)
            if  probabilistic_support >= min_sup:
                
                WFIS_k.add(X)
        
        WFIS.update(WFIS_k)
        WFIS_1 = WFIS_k
        k += 1
    
    result = []
    WFIS = sorted(WFIS, key= lambda x: len(x), reverse=True)
    
    for x in WFIS:
        isContain = False
        for y in result:
            if x.issubset(y):
                isContain = True
                break
        if not isContain:
            result.append(x)
    
    return result


# Example

In [6]:
# Example 1
D = [
    {'A': 0.6, 'B': 0.7},
    {'A': 0.2, 'C': 0.3},
]

weighted = {
    'A': 1,
    'B': 1,
    'C': 1
}

min_sup = 1
wfpt = 0.1

# #Example 2
# #Example 2
D = [
    {"1": 0.5, "2": 0.7, "4": 0.8, "5": 0.9},
    {"2": 0.6, "3": 0.8, "4": 0.6, "5": 0.8},
    {"3": 0.6, "4": 0.9, "5": 0.5},
    {"1": 0.6, "3": 0.7, "4": 0.8, "5": 0.8},
    {"1": 0.8, "2": 0.9, "3": 0.5, "4": 0.6, "5": 0.7},
    {"2": 0.6, "4": 0.9, "5": 0.8},
]

weighted = {
    "1": 0.3,
    "2": 0.9,
    "3": 0.5,
    "4": 0.6,
    "5": 0.9
}

min_sup = 2
wfpt = 0.2
# print(candidate_generate_expected_bound(D, min_sup, wfpt))
print(WD_FIM(D, weighted, min_sup, wfpt))

frozenset({'2'}) - 4
frozenset({'5'}) - 5
frozenset({'4'}) - 5
frozenset({'1'}) - 2
frozenset({'3'}) - 3
ck {frozenset({'5', '1'}), frozenset({'3', '2'}), frozenset({'5', '2'}), frozenset({'2', '1'}), frozenset({'3', '1'}), frozenset({'4', '1'}), frozenset({'4', '5'}), frozenset({'3', '5'}), frozenset({'4', '3'}), frozenset({'4', '2'})}
nc_k set()
rc_k {frozenset({'5', '1'}), frozenset({'3', '2'}), frozenset({'5', '2'}), frozenset({'2', '1'}), frozenset({'3', '1'}), frozenset({'4', '1'}), frozenset({'4', '5'}), frozenset({'3', '5'}), frozenset({'4', '3'}), frozenset({'4', '2'})}
ck {frozenset({'4', '2', '1'}), frozenset({'5', '2', '1'}), frozenset({'3', '5', '1'}), frozenset({'4', '3', '5'}), frozenset({'4', '3', '1'}), frozenset({'3', '5', '2'}), frozenset({'4', '3', '2'}), frozenset({'4', '5', '2'}), frozenset({'4', '5', '1'})}
nc_k {frozenset({'5', '2', '1'}), frozenset({'5', '3', '2'}), frozenset({'5', '3', '1'}), frozenset({'4', '3', '1'}), frozenset({'3', '2', '1'}), frozenset({'

# 4. Real dataset

In [ ]:
def read_dataset(file_path, mean, variance):
    uncertain_database = []

    # Initialize normal distribution
    random.seed(12345)

    try:
        with open(file_path, 'r') as file:
            curr_id_transaction = 0
            count_num_line = 1

            for line in file:
                data_line_transaction = line.strip().split(" ")
                cur_transaction = {}

                for item in data_line_transaction:
                    data_item = item.split("-")
                    value = data_item[0]
                    prob = float(data_item[1])
                    cur_transaction[value] = prob

                uncertain_database.append(cur_transaction)
                curr_id_transaction += 1
                count_num_line += 1
    except IOError as e:
        print(e)

    print(uncertain_database)

    return uncertain_database

def generate_weighted_table(uncertain_database):
    weighted_table = {}
    distinct_itemset_database = discover_single_items(uncertain_database)

    random.seed(12345)

    for item in distinct_itemset_database:
        weighted = round(random.random(), 1)
        if weighted <= 0:
            weighted = 0.1
        else:
            if weighted >= 1:
                weighted = 0.9
        weighted_table[item] = weighted

    return weighted_table

## Dataset

In [ ]:
path = './data/accidents_10K.data'
path = './data/connect4_10K.data'
path = './data/T40I10D100K_10K.data'
path = './data/us_10K.data'
D = read_dataset(path, 0.78, 0.65)
W = generate_weighted_table(D)

for i in range(0, 5):
    print(D[i])

[{'1': 0.4, '2': 0.7, '3': 0.8, '4': 0.3, '5': 0.1, '6': 0.3, '7': 0.2, '8': 0.9, '9': 0.3, '10': 0.7, '11': 1.0, '12': 0.5, '13': 0.1, '14': 0.9, '15': 0.7, '16': 0.4, '17': 0.2, '18': 0.5, '19': 0.2, '20': 0.3, '21': 0.5, '22': 0.4, '23': 0.4, '24': 0.7, '25': 0.3, '26': 0.0, '27': 0.2, '28': 0.6, '29': 0.1, '30': 0.5, '31': 0.6}, {'2': 0.8, '5': 0.5, '7': 0.7, '8': 0.1, '9': 0.9, '10': 1.0, '12': 0.5, '13': 0.8, '14': 0.9, '15': 0.6, '16': 0.2, '17': 0.4, '18': 0.1, '20': 0.2, '22': 0.1, '23': 0.9, '24': 0.9, '25': 0.1, '27': 0.7, '28': 0.9, '29': 1.0, '32': 0.9, '33': 0.8, '34': 0.6, '35': 0.9, '36': 1.0, '37': 0.6, '38': 0.4, '39': 0.8}, {'7': 0.6, '10': 0.7, '12': 0.9, '13': 0.9, '14': 0.1, '15': 0.1, '16': 0.3, '17': 0.1, '18': 0.5, '20': 0.3, '25': 0.8, '28': 0.6, '29': 0.1, '30': 0.9, '33': 0.2, '40': 0.5, '41': 0.5, '42': 0.3, '43': 0.5, '44': 0.7, '45': 0.1, '46': 0.4, '47': 0.4, '48': 0.7, '49': 0.5, '50': 0.4, '51': 0.9, '52': 0.8}, {'1': 0.3, '5': 0.6, '8': 0.6, '10': 0.6

## Min support and minconfidence

In [ ]:
min_sup = 0.1*10000
min_conf = 0.7

# 5. Run algorithm

In [ ]:
# Start the timer
start_time = time.time()

# Your code here
result = algorithms(D, W, min_sup, min_conf)
print(result)
print(len(result))

# End the timer
end_time = time.time()

# Calculate the runtime
runtime = end_time - start_time
print(f"Runtime: {runtime} seconds")


C:\Users\hung.tt\AppData\Local\Temp\ipykernel_392\2239993570.py:2: RuntimeWarning: invalid value encountered in sqrt
  return (2*min_sup - np.log(min_conf) - np.sqrt((np.log(min_conf))**2 - 8*min_sup*np.log(min_conf))) / 2


[['1', '22'], ['1', '41'], ['1', '25'], ['1', '66'], ['22', '41'], ['22', '25'], ['22', '38'], ['22', '66'], ['41', '25'], ['41', '38'], ['41', '30'], ['41', '66'], ['25', '38'], ['10', '25'], ['25', '66'], ['38', '66'], ['23', '66'], ['10', '66'], ['30', '66'], ['80'], ['5'], ['82']]
22
Runtime: 16.93607759475708 seconds
